In [8]:
from dataclasses import dataclass

# from langchain.chat_models import init_chat_model
from langchain_community.chat_models.tongyi import ChatTongyi
from langgraph.graph import StateGraph, START


@dataclass
class MyState:
    topic: str
    joke: str = ""


# model = init_chat_model(model="gpt-4o-mini")
model = ChatTongyi( # type: ignore
    model="qwen3-0.6b",
    model_kwargs={
        "temperature": 0.0,
        "enable_thinking": False,
    },
    streaming=True,  # 手动启用流式返回
)

def call_model(state: MyState):
    """Call the LLM to generate a joke about a topic"""
    # Note that message events are emitted even when the LLM is run using .invoke rather than .stream
    model_response = model.invoke(  
        [
            {"role": "user", "content": f"Generate a joke about {state.topic} more than 1000 words."}
        ]
    )
    return {"joke": model_response.content}

graph = (
    StateGraph(MyState)
    .add_node(call_model)
    .add_edge(START, "call_model")
    .compile()
)

In [9]:
# The "messages" stream mode returns an iterator of tuples (message_chunk, metadata)
# where message_chunk is the token streamed by the LLM and metadata is a dictionary
# with information about the graph node where the LLM was called and other information
for message_chunk, metadata in graph.stream(
    {"topic": "ice cream"},
    stream_mode="messages",  
):
    if message_chunk.content:
        print(message_chunk.content, end="|", flush=True)

###| The| Ice| Cream| Adventure

It was a sunny afternoon in the town of Maplewood, where the| air was thick with the scent of fresh bread and warm coffee. As the sun| dipped below the horizon, the community gathered for a local ice cream festival that had| been planned for weeks. The event was a fun-filled day for families, friends|, and even the kids who loved to play outside.

The festival started at| 3 p.m., and as the crowd began to gather, it was clear that| everyone wanted to try something new. There were ice cream trucks selling their wondrous| treats, each one carrying a variety of flavors—sweet, salty, and even| some sweet-and-salty combinations. The sight of colorful cones and scoops of| different flavors made the atmosphere feel lively and exciting.

As the day progressed, the| crowd became more enthusiastic, and people started to order their favorite ice creams. Some| ordered just a scoop, while others wanted a whole cone. The excitement was palp|able, and the laughter ech